## Custom Segmentation Algorithm Profling

Notebook to profile segmentation implementation ([spheroid_cytometer.py](spheroid_cytometer.py)) for performance deficiencies.

In [1]:
%matplotlib inline
%run spheroid_cytometer.py
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from scipy import ndimage
from skimage import feature
from scipy import ndimage
from skimage import morphology
from skimage import filters
from cytokit.ops import cytometry as cytometer_op
from cytokit.ops import tile_generator
from cytokit import math as ck_math
from cytokit import config as ck_config
from itkwidgets import view

In [2]:
exp_name = '20190215-mc38-dmso-control'
exp_dir = 'XY07'
data_dir = osp.join(os.environ['CYTOKIT_DATA_DIR'], 'cytokit', 'mc38-spheroid', exp_name, exp_dir, 'raw')
config_dir = osp.join(os.environ['CYTOKIT_REPO_DIR'], 'pub', 'config', 'mc38-spheroid')
config = ck_config.load(config_dir)
os.environ['CYTOKIT_PATH_FORMATS'] = 'get_default_path_formats("1_' + exp_dir + '_{tile:05d}_Z{z:03d}_CH{channel:d}.tif")'
config.register_environment()

In [3]:
tile_gen = tile_generator.CytokitTileGenerator(config, data_dir, 0, 4)

In [ ]:
tile = tile_gen.run()
tile.shape

In [19]:
from skimage import transform
img = tile[0, :, 0]
#img = transform.rescale(img, (1, .5, .5), multichannel=False, preserve_range=True, anti_aliasing=True, mode='reflect').astype(img.dtype)
img.shape, img.dtype

((26, 1440, 1920), dtype('uint16'))

### Debugging

In [20]:
view(img)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageUS3; proxy of <Swig Object of type 'itkIm…

In [777]:

def flat_ball(r, rz):
    return np.stack([
        np.pad(morphology.disk(rz), r-rz, 'constant'),
        morphology.disk(r),
        np.pad(morphology.disk(rz), r-rz, 'constant'),
    ])

def prep(img):
    img = ndimage.median_filter(img, size=(1, 3, 3))
    img = img_as_float(img)
    return img

#gimage = standardize(img_prep, 16)
#gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
gimage = prep(img)

gimage = gimage - ndimage.gaussian_filter(gimage, sigma=(.1, 1, 1), mode='reflect')
#gimage = gimage - ndimage.gaussian_filter(gimage, sigma=(0, 16, 16), mode='reflect')
gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
gimage = ndimage.generic_gradient_magnitude(gimage, ndimage.sobel)
print(gimage.min(), gimage.max(), len(np.unique(gimage)))
gimage = ndimage.gaussian_filter(gimage, sigma=(.2, 2, 2), mode='constant')
#gimage = np.stack([filters.sobel(gimage[i]) for i in range(gimage.shape[0])])
gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
print(gimage.min(), gimage.max(), len(np.unique(gimage)))
mimage = gimage > filters.threshold_otsu(gimage)

selem = ndimage.generate_binary_structure(3, 1)
selem[0] = False
selem[2] = False
mimage = ndimage.binary_closing(mimage, structure=selem, iterations=1)
mimage = ndimage.binary_fill_holes(mimage, structure=selem)
mimage = morphology.remove_small_objects(mimage, min_size=512)
limage, nlabel = ndimage.label(mimage)
print('Num objects:', nlabel)

dimage = ndimage.distance_transform_edt(mimage, sampling=(10, 1, 1))
dimage = ndimage.gaussian_filter(dimage, sigma=(.5, 5, 5), mode='constant')

pimage = feature.peak_local_max(dimage, exclude_border=False, min_distance=100, indices=False, labels=limage)
pimage = ndimage.label(pimage)[0]
#pimage = ndimage.binary_dilation(pimage, iterations=3)

simage = segmentation.watershed(-dimage, pimage, mask=mimage)


0.00025709607953490157 7.499200406210606 17971200
0.0 1.0 17971200
Num objects: 33


In [778]:
nlabel

33

In [779]:
#gimage = exposure.rescale_intensity(gimage.astype(np.float32), (0, 1))
gimage.min(), gimage.max(), len(np.unique(gimage))

(0.0, 1.0, 17971200)

In [780]:
simage.min(), simage.max()

(0, 33)

In [781]:
#view(dimage.astype(np.float32))
view(simage.astype(np.float32))

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [604]:
img_bound = np.stack([segmentation.find_boundaries(gimage[i]) for i in range(gimage.shape[0])])
view(img_bound.astype(np.float32) * .5 + img_as_float(img).astype(np.float32))

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

### Profiling

In [ ]:
op = cytometer_op.Cytometry2D(config)

In [ ]:
op.initialize()

In [ ]:
# %%time
# img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=None, min_object_size=1024, min_peak_dist=200)
# img_seg.shape

In [ ]:
%%time
img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=.5, min_object_size=512, min_peak_dist=75)
img_seg.shape

In [8]:
#view(img_seg[:, 5].astype(np.float32).copy())

In [10]:
%%time
stats = op.cytometer.quantify(tile, img_seg, channel_names=config.channel_names, 
                              cell_graph=True, morphology_features=False, 
                              cell_intensity=['mean', 'sum'])

CPU times: user 4min 23s, sys: 1.87 s, total: 4min 25s
Wall time: 4min 25s


In [11]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 26 columns):
id                     371 non-null int64
x                      371 non-null float64
y                      371 non-null float64
z                      371 non-null int64
cm:size                371 non-null int64
cm:diameter            371 non-null float64
cm:perimeter           371 non-null float64
cm:solidity            371 non-null float64
cm:circularity         371 non-null float64
nm:size                371 non-null int64
nm:diameter            371 non-null float64
nm:perimeter           371 non-null float64
nm:solidity            371 non-null float64
nm:circularity         371 non-null float64
cg:n_neighbors         371 non-null int64
cg:neighbor_ids        371 non-null object
cg:adj_neighbor_pct    371 non-null object
cg:adj_bg_pct          371 non-null object
ci:BF:mean             371 non-null float64
ci:LIVE:mean           371 non-null float64
ci:DEAD:mean           371 n

In [12]:
stats['id'].nunique()

23

In [15]:
#!pip install line_profiler
%load_ext line_profiler

In [18]:
%lprun -f op.cytometer.quantify stats = op.cytometer.quantify(tile, img_seg, channel_names=config.channel_names, cell_graph=True, morphology_features=False, cell_intensity=['mean', 'sum'])
#%lprun -f op.cytometer.segment img_seg = op.cytometer.segment(tile[0, :, 0])

Timer unit: 1e-06 s

Total time: 265.727 s
File: /lab/repos/cytokit/pub/analysis/mc38-spheroid/spheroid_cytometer.py
Function: quantify at line 131

Line #      Hits         Time  Per Hit   % Time  Line Contents
   131